# WoRMS Query
### 5/11/22

[WoRMS API Documentation](https://www.marinespecies.org/aphia.php?p=webservice)


In [ ]:
%pip install requests
import requests

## By Scientic Name
/AphiaIDByName/{ScientificName}


In [ ]:
import urllib.parse

def get_worms_record_precise(scientific_name):
    parsed_name = urllib.parse.quote(scientific_name)
    url = "https://www.marinespecies.org/rest/AphiaIDByName/" + parsed_name
    response = requests.get(url)
    return response.json
    

## Fuzzy Search
Useful if the exact match can't be found!

/AphiaRecordsByMatchNames

Input:
scientificnames[] (array)
marine_only (bool)

In [26]:
import urllib.parse

""" Returns the Aphia Records from WoRMS that match the provided scientific names, using fuzzy search.
"""
def get_worms_record_fuzzy(names: list):
    # handle case where name is a single string
    if type(names) == "str":
        names = [names]

    url = "https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?{}&marine_only=true"
    # List out scientific names as URL parameters
    parsed_names = list(map(urllib.parse.quote, names))
    print(parsed_names)
    names_query = "scientificnames[]=" + "&scientificnames[]=".join(parsed_names)
    # Escape characters and insert into URL
    url = url.format(names_query)
    print(url)

    response = requests.get(url)
    return response
    
    
example_names = ["Porlia rufescens"]  # deliberately misspelled :)
response = get_worms_record_fuzzy(example_names)
print(response.status_code)
print(response.json())

['Porlia%20rufescens', 'a']
https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames[]=Porlia%20rufescens&scientificnames[]=a&marine_only=true
200
[[{'AphiaID': 135310, 'url': 'https://www.marinespecies.org/aphia.php?p=taxdetails&id=135310', 'scientificname': 'Poralia rufescens', 'authority': 'Vanhöffen, 1902', 'status': 'accepted', 'unacceptreason': None, 'taxonRankID': 220, 'rank': 'Species', 'valid_AphiaID': 135310, 'valid_name': 'Poralia rufescens', 'valid_authority': 'Vanhöffen, 1902', 'parentNameUsageID': 135267, 'kingdom': 'Animalia', 'phylum': 'Cnidaria', 'class': 'Scyphozoa', 'order': 'Semaeostomeae', 'family': 'Ulmaridae', 'genus': 'Poralia', 'citation': 'Collins, A. G.; Jarms, G.; Morandini, A. C. (2022). World List of Scyphozoa. Poralia rufescens Vanhöffen, 1902. Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=135310 on 2022-05-12', 'lsid': 'urn:lsid:marinespecies.org:taxname:135310', 'isMarin

In [41]:
""" Tries to get the Aphia Record for a given identification name from the
World Register of Marine Species (WoRMS).
Searches first by exact scientific name, and then with fuzzy search.

Returns None if no match was found.
"""
def try_get_worms_record(name):
    # Escape non-URL characters in name
    parsed_name = urllib.parse.quote(name)

    # First try to access using the scientific name.
    url = "https://www.marinespecies.org/rest/AphiaRecordsByName/" + parsed_name
    response = requests.get(url)
    if response.status_code == 200:
        # Success! 
        # Response may return multiple Aphia records, so we return the first one.
        return response.json()[0]

    elif response.status_code == 204:
        # No records found, so we try again with fuzzy search.
        url = "https://www.marinespecies.org/rest/AphiaRecordsByMatchNames?scientificnames[]={}&marine_only=true"
        url = url.format(parsed_name)
        response = requests.get(url)
        if (response.status_code == 200): # Success!
            return response.json()[0][0]   

    if response.status_code == 400:  # Check if either response was faulty.
        # Likely an issue with query, so we raise an error.
        raise RuntimeError("Response status code was 400 for url '{}'. Parameters may be invalid.".format(url))
    
    return None

print("Result with Poralia rufescens:")
print(try_get_worms_record("Poralia rufescens"))
print("\nResult with 'Pralia rufcens':")
print(try_get_worms_record("Pralia rufcens"))




Result with Poralia rufescens:
{'AphiaID': 135310, 'url': 'https://www.marinespecies.org/aphia.php?p=taxdetails&id=135310', 'scientificname': 'Poralia rufescens', 'authority': 'Vanhöffen, 1902', 'status': 'accepted', 'unacceptreason': None, 'taxonRankID': 220, 'rank': 'Species', 'valid_AphiaID': 135310, 'valid_name': 'Poralia rufescens', 'valid_authority': 'Vanhöffen, 1902', 'parentNameUsageID': 135267, 'kingdom': 'Animalia', 'phylum': 'Cnidaria', 'class': 'Scyphozoa', 'order': 'Semaeostomeae', 'family': 'Ulmaridae', 'genus': 'Poralia', 'citation': 'Collins, A. G.; Jarms, G.; Morandini, A. C. (2022). World List of Scyphozoa. Poralia rufescens Vanhöffen, 1902. Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=135310 on 2022-05-12', 'lsid': 'urn:lsid:marinespecies.org:taxname:135310', 'isMarine': 1, 'isBrackish': 0, 'isFreshwater': 0, 'isTerrestrial': 0, 'isExtinct': 0, 'match_type': 'like', 'modified': '2019-12-16T00:13:05.090

In [42]:
try_get_worms_record("Edwardsiidae")

{'AphiaID': 100665,
 'url': 'https://www.marinespecies.org/aphia.php?p=taxdetails&id=100665',
 'scientificname': 'Edwardsiidae',
 'authority': 'Andres, 1881',
 'status': 'accepted',
 'unacceptreason': None,
 'taxonRankID': 140,
 'rank': 'Family',
 'valid_AphiaID': 100665,
 'valid_name': 'Edwardsiidae',
 'valid_authority': 'Andres, 1881',
 'parentNameUsageID': 888373,
 'kingdom': 'Animalia',
 'phylum': 'Cnidaria',
 'class': 'Anthozoa',
 'order': 'Actiniaria',
 'family': 'Edwardsiidae',
 'genus': None,
 'citation': 'Daly, M.; Fautin, D. (2022). World List of Actiniaria. Edwardsiidae Andres, 1881. Accessed through: World Register of Marine Species at: https://www.marinespecies.org/aphia.php?p=taxdetails&id=100665 on 2022-05-12',
 'lsid': 'urn:lsid:marinespecies.org:taxname:100665',
 'isMarine': 1,
 'isBrackish': 1,
 'isFreshwater': None,
 'isTerrestrial': 1,
 'isExtinct': None,
 'match_type': 'like',
 'modified': '2016-10-22T18:09:45.233Z'}

Partially concerned by non-organism labels:

![./non-organism/id.jpg]